# Preamble

**BEFORE RUNNING THIS CODE: **Download the launch 12 GPS data [here](https://github.com/psas/Launch-12/blob/gh-pages/data/GPS/JGPS%40-32.041913222), and place it in the */resources* folder.

This notebook demonstrates the satellite acquisition proccess, in which we determine which satellites are overhead. This code implements the parallel code phase search algorithm, which is the most complex of those available, but is also the fastest since it returns a result in just one iteration. It is possible that we will need to use a different algorithm when the code is running on an embedded platform.

**Note: we should only perform this acquisition method on ~20ms sections of data, see book for details**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import math

##Grabs the GoldCode module from the root of the repo
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from GoldCode import GoldCode

# Start of ByteToIQPairs Function
def ByteToIQPairs( TheByte ):
    IQPairs = []
    
    # This code reads each of the four pairs of bits from the byte 
    # and determines the sign and magnitude. Then it returns a list 
    # containing two pairs of IQ data as floating point [I1,Q1,I2,Q2].
    # For magnitude: a bit value of 1 means mag 1, 0 means mag 1/3
    # For sign: a bit value of 1 means negative, 0 means positive
    # This interpretation was taken by the sample code provided
    # in the PSAS Launch12 github repo (example was provided in C)
    #
    # f_s = 4.092 MHz (sampling rate of raw signal)
   
    IMag1 = (TheByte >> 7) & (0b00000001)
    ISign1 = (TheByte >> 6) & (0b00000001)
    I1 = 1.0 if (IMag1 == 1) else 1.0/3.0
    I1 = -I1 if (ISign1 == 1) else I1
    IQPairs.append(I1)
    
    QMag1 = (TheByte >> 5) & (0b00000001)
    QSign1 = (TheByte >> 4) & (0b00000001)
    Q1 = 1.0 if (QMag1 == 1) else 1.0/3.0
    Q1 = -Q1 if (QSign1 == 1) else Q1
    IQPairs.append(Q1)    
        
    IMag2 = (TheByte >> 3) & (0b00000001)
    ISign2 = (TheByte >> 2) & (0b00000001)
    I2 = 1.0 if (IMag2 == 1) else 1.0/3.0
    I2 = -I2 if (ISign2 == 1) else I2
    IQPairs.append(I2)    
        
    QMag2 = (TheByte >> 1) & (0b00000001)
    QSign2 = (TheByte >> 0) & (0b00000001)
    Q2 = 1.0 if (QMag2 == 1) else 1.0/3.0
    Q2 = -Q2 if (QSign2 == 1) else Q2
    IQPairs.append(Q2)
    
    return IQPairs 
# End of ByteToIQPairs Function


# Import data File

In [ ]:
# Beginning of main code
# Open data file
# Information on file "JGPS@-32.041913222" can be found at: 
# https://github.com/psas/Launch-12/tree/gh-pages/data/GPS
print("Opening a file.")
fHandle = open('../resources/JGPS@-32.041913222','r', encoding = 'utf-8', errors = 'ignore')
print("File handle is: %d." % (fHandle.fileno()))

# Read file one byte at a time, extract the two 
# IQ pairs, and store in array, after conversion to float.
# Will use 1ms in order to avoid duplication of C/A code
# See page 85 of "A Software-Defined GPS and Galileo Receiver A Single-Frequency Approach"
fs = 4.092*10**6 # Sampling Frequency [Hz]
Ts = 1/fs # Sampling Period [s]
SampleLength = 5*10**(-3) # 20 ms Sample length
StartingByte = 0 # Can change this if we want to discard initial samples
TotalSamples = int(np.ceil(SampleLength/Ts)) 
TotalBytes = int(np.ceil(TotalSamples/2))
print("Total Samples to read: %d"%(TotalSamples))
print("Total Bytes read: %d." %(TotalBytes))
print("Which equals %d IQ pairs." %(TotalBytes*2))
print("Sample Length: %f seconds." %(TotalBytes*2*Ts))

i = StartingByte
SingleByte = fHandle.read(1)
IData = []
QData = []
while SingleByte != "":
    IQPairs = ByteToIQPairs(ord(SingleByte))
    IData.append(IQPairs[0])
    IData.append(IQPairs[2])
    QData.append(IQPairs[1])
    QData.append(IQPairs[3])
    #print("I: %f, Q: %f ." % (IQPairs[0], IQPairs[1]))
    #print("I: %f, Q: %f ." % (IQPairs[2], IQPairs[3]))
    
    i += 1
    if i >= (TotalBytes - StartingByte):
        break
    SingleByte = fHandle.read(1)

fHandle.close()
print("File is now closed.")

# Convert to complex array

In [ ]:
print("length of IData is: %d" % (len(IData)))
print("Length of QData is: %d" % (len(QData)))

CData = np.zeros(len(IData), dtype=np.complex)

for d in range(len(IData)):
    CData[d] = IData[d] + QData[d] * 1j
    
#print(CData)

# LO Sweep

- Between these 2 steps we need to sweep the local oscillator due to Doppler effect
- Pick a frequency, and run the steps from here assuming the Doppler shift puts us @ that freq
- We need to multiply the incoming data by a complex sine wave 
- The frequency of the generated wave is equal to the difference between the actual and target frequencies
- The sine wave must be complex in order to get a negative frequency (sweep both directions)
- Book recommends +/- 10kHz in 500Hz steps (pg 77)

# Take fft

In [ ]:
# Since we are using circular-convolution, nfft must be equal to the number of samples
nfft = TotalSamples
print("Calculated deltaFreq: %f [Hz]" %(fs/nfft))

# Generate frequency lists for plotting fft
# Can use f for regular fft and fshift for shifted version
fs_kHz = fs/1000 #Make easier to read
f = np.linspace (0,fs_kHz,nfft, endpoint=False)
fshift = np.linspace(-fs_kHz/2,fs_kHz/2,nfft,endpoint=False)

# Reference data size section of book to pick the len param. Should be based sample rate
ffC = np.fft.fft(CData, nfft)

# Plot output

In [ ]:
plt.plot(f, np.abs(ffC))

plt.title("FFT of Complex Data")
plt.xlabel("Frequency (kHz)")
plt.show()

# Get a Gold Code

- Gold Code must be generated as though it were sampled at the sampling rate
- 1023 chips that must take 1ms to pass through (due to chipping rate)
- We are sampling @ 4.092 MHz
- So we need the GC to repeat each chip 4 times before going to the next chip.

- The numpy.repeat() function does this exactly

In [ ]:
#Satellites in the sky during launch:
#SV:  1   25.5° el    71.5° az
#SV:  4   14.5° el    46.1° az
#SV:  6   -2.5° el   172.8° az
#SV:  7   11.1° el   130.3° az
#SV: 11   24.3° el    56.3° az
#SV: 13   46.5° el   244.3° az
#SV: 15   32.1° el   287.7° az
#SV: 17   62.3° el   189.2° az
#SV: 24    5.7° el   310.0° az
#SV: 28   65.0° el    41.4° az
#SV: 30   42.6° el   127.9° az


#Choose which satellite's C/A code is generated
Satellite = 13

# Create list of C/A code Taps, for simpler sat selection",
sat = [(2,6),(3,7),(4,8),(5,9),(1,9),(2,6),(1,8),(2,9),(3,10),(2,3),(3,4),(5,6),(6,7),(7,8),(8,9),(9,10),(1,4),(2,5),(3,6),(4,7),(5,8),(6,9),(1,3),(4,6),(5,7),(6,8),(7,9),(8,10),(1,6),(2,7),(3,8),(4,9),(5,10),(4,10),(1,7),(2,8),(4,10)]

# Create Code Generator object for chosen Satellite
CodeGen = GoldCode(sat[Satellite - 1]) # Index starts at zero

# Generate CA Code
CACode = CodeGen.getCode(1023)

# Reformat GC to have -1 in place of 0
for n,i in enumerate(CACode):
    if i==0:
        CACode[n]=-1
        
# Repeat each chip 4 times (See markdown in above cell), to match our ADC sample frequency",
CACodeSampled = np.repeat(CACode,4)
print("Satellite chosen: %d, with tap: %s" %(Satellite,str(sat[Satellite - 1])))

In [ ]:
Codefft = np.fft.fft(CACodeSampled, nfft)

plt.plot(f, np.abs(Codefft))
plt.title("FFT of Gold Code")
plt.xlabel("Frequency (kHz)")
plt.show()

# Complex Conjugate of Gold Code fft

real(fft) - imag(fft)j

In [ ]:
GCConj = np.conjugate(Codefft)

result = np.fft.ifft(GCConj * ffC,nfft)

resultSQ = np.abs(result) ** 2
resultLog = -10*np.log10(resultSQ)

t = np.linspace(0,SampleLength,nfft,endpoint=True) 
print("Length of t: %d" %(len(t)))
print("Length of IFFT: %d" %(len(resultSQ)))

plt.plot(t,resultLog)
plt.title("Cross-correlation without adjusting for doppler shift.")
plt.xlabel("Time (seconds)")
plt.show()

print("Max Value: %f, at freqshift: %d, with index %d"%(np.max(resultLog),0,np.argmax(resultLog)))

# Notes

- From book appendix: we should look for the top 2 peaks, and use the ratio to determine whether there is a signal or just noise
- We should get the C code running and be sure to compare results. Acquisition works in the C code, but tracking is untested

## Shift frequency using multiplication by a complex exponential

In [ ]:
freqShift = 3195 #[Hz]

# Initialize complex array
CDataShifted = np.zeros(len(CData), dtype=np.complex)
print(len(CData))
print(len(CDataShifted))
print(len(t))
for i in range(len(CData)):
    CDataShifted[i] = CData[i]*np.exp(1j*2*np.pi*freqShift*t[i]) + CData[i]*np.exp(1j*2*np.pi*freqShift*t[i])
    
fftCDataShifted = np.fft.fft(CDataShifted,nfft)

result = np.fft.ifft(GCConj * fftCDataShifted,nfft)

resultSQ = np.abs(result) ** 2
resultLog = -10*np.log10(resultSQ)

plt.plot(t,resultLog)
plt.title("Cross-correlation with doppler adjustment of %d [Hz]"%(freqShift))
plt.xlabel("Time (seconds)")
plt.show()

print("Max Value: %f, at freqshift: %d, with index: %d div/4: %f"%(np.max(resultLog),freqShift,np.argmax(resultLog),np.argmax(resultLog)/4))

In [ ]:
# Shift gold code by maximum index from prior cell
CACode_Shifted = CACodeSampled
np.roll(CACode_Shifted,np.argmax(resultLog))

print(len(CData))
print(len(CACode_Shifted))
# Multiply by frequency shifted complex data: 
Result = np.zeros(len(CData), dtype=np.complex)
for i in range(len(CData)):
    Result[i] = CACode_Shifted[i]*CData[i]

plt.plot(t,Result)
plt.show()

In [ ]:
t

In [ ]:
len(t)